#  Module 5: Réaliser un backtesting et mesurer la performance en python


Le **backtesting** consiste à tester une stratégie d’investissement sur des **données historiques** pour évaluer sa performance passée. Cela permet de :

- Vérifier la robustesse d'une stratégie
- Ajuster les paramètres sans risque
- Comprendre la dynamique du marché


Ce cours a pour objectifs de :

- Comprendre le principe du backtesting et son intérêt en finance.
- Implémenter une stratégie de trading simple en Python (ex : croisement de moyennes mobiles).
- Simuler des transactions à l’aide de la librairie `backtrader`.
- Mesurer et interpréter les performances de la stratégie (rendement, capital final, drawdown).
- Manipuler des outils Python adaptés à l’analyse financière : `pandas`, `matplotlib`, `yfinance`, `backtrader`.


In [ ]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt

# Télécharger les données Apple
df = yf.download('AAPL', start='2020-01-01', end='2024-12-31')


# Ça fonctionne même si yfinance renvoie un DataFrame multi-niveaux
df.columns = df.columns.droplevel('Ticker')
df.dropna(inplace=True)
df.head()


## 1. Implémenter une stratégie simple

Une stratégie de **croisement de moyennes mobiles** est l’une des méthodes les plus populaires et accessibles en trading algorithmique.

### Principe

Cette stratégie repose sur la comparaison de deux moyennes mobiles :

- **SMA courte** (ex : 20 jours) : réagit plus vite aux variations du marché.
- **SMA longue** (ex : 50 jours) : filtre les mouvements de court terme et donne une tendance plus stable.

### Règles de la stratégie

- **Signal d'achat** : lorsque la SMA courte **croise au-dessus** de la SMA longue → début d'une tendance haussière probable.
- **Signal de vente** : lorsque la SMA courte **croise en dessous** de la SMA longue → début d'une tendance baissière probable.

Cette stratégie permet de suivre la tendance du marché sans tenter de prédire les retournements.

> L'objectif est de **laisser courir les gains** et de **limiter les pertes** en réagissant uniquement aux signaux donnés par les croisements.

### Avantages

- Facile à comprendre et à coder
- Adaptable à n’importe quel actif
- Fonctionne bien dans les marchés en tendance

### Limites

- Moins performant dans les marchés en range (sans tendance)
- Peut générer des faux signaux (bruit de marché)


In [2]:
# 2. Calcul des moyennes mobiles
df['SMA20'] = df['Close'].rolling(window=20).mean()
df['SMA50'] = df['Close'].rolling(window=50).mean()



In [3]:
# 3. Détection des signaux
df['Signal'] = 0
df.loc[df['SMA20'] > df['SMA50'], 'Signal'] = 1
df.loc[df['SMA20'] < df['SMA50'], 'Signal'] = 0
df['Position'] = df['Signal'].diff()

In [ ]:

df[['Close', 'SMA20', 'SMA50', 'Signal', 'Position']].tail()

In [ ]:

# 4. Tracer le graphique
plt.figure(figsize=(14, 8))
plt.plot(df['Close'], label='Cours de clôture', alpha=0.5)
plt.plot(df['SMA20'], label='SMA 20 jours', color='green')
plt.plot(df['SMA50'], label='SMA 50 jours', color='red')

# Points d'achat
plt.plot(df[df['Position'] == 1].index, df['Close'][df['Position'] == 1], '^', color='green', markersize=10, label='Achat')

# Points de vente
plt.plot(df[df['Position'] == -1].index, df['Close'][df['Position'] == -1], 'v', color='red', markersize=10, label='Vente')

plt.title("Stratégie de croisement de moyennes mobiles - AAPL")
plt.xlabel("Date")
plt.ylabel("Prix")
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()

## 2- Simuler les transactions avec Backtrader


[**Backtrader**](https://www.backtrader.com/) est une des bibliothèques Python les plus puissantes pour :
- Créer des **stratégies personnalisées**
- Simuler **les ordres (buy/sell)**
- Gérer **le portefeuille virtuel**
- Obtenir des **statistiques détaillées** (rendement, drawdown, winrate, etc.)


###  Comment fonctionne une simulation Backtrader ?

1. Charger les données (`yfinance`, CSV ou API)
2. Définir la **stratégie** (ex : croisement de moyennes mobiles)
3. Initialiser le **portefeuille virtuel** avec un capital de départ
4. Lancer le **backtest**
5. Observer les résultats :
   - Rendement net
   - Pertes maximales (drawdown)
   - Ratio de trades gagnants
   - Nombre de transactions


### Interprétation des résultats

- **Courbe de capital (equity curve)** : doit être croissante et régulière
- **Signal d’achat vs performance réelle** : valide ou invalide la stratégie
- **Drawdown** : mesure la **perte maximale** en cas de mauvaise passe (à surveiller)
- **Sharpe ratio** : compare rendement/risque → plus il est haut, mieux c’est



###  Attention aux illusions de performance

Même si une stratégie fonctionne bien sur le passé, cela ne garantit **pas sa performance future**.

- Les données peuvent être **sur-optimisées** (overfitting)
- Le marché peut **changer de régime**
- Des **frais de transaction** ou des latences peuvent modifier les résultats



In [6]:
!pip install backtrader --quiet #

In [ ]:
import backtrader as bt
# On ne garde que les colonnes nécessaires pour le trading
data = df[['Open', 'High', 'Low', 'Close', 'Volume']].copy()
data.dropna(inplace=True)  # On supprime les lignes incomplètes

# Backtrader préfère les noms de colonnes en minuscules
data.columns = [col.lower() for col in data.columns]

# On remet l'index 'Date' sous forme de colonne pour Backtrader
data.reset_index(inplace=True)
data.reset_index(inplace=True)
data.head()

In [ ]:

# ----------------------------
#  2. Créer un "Data Feed" pour Backtrader
# ----------------------------

# Un "Data Feed", c’est un connecteur entre tes données pandas et Backtrader.
# Il lui explique comment lire la colonne 'Date', 'Open', etc. depuis le DataFrame.
class PandasData(bt.feeds.PandasData):
    params = (
        ('datetime', 'Date'),        # Indique la colonne utilisée comme date
        ('open', 'open'),
        ('high', 'high'),
        ('low', 'low'),
        ('close', 'close'),
        ('volume', 'volume'),
        ('openinterest', -1),        # Pas utilisé ici, on le désactive avec -1
    )


# ----------------------------
# 📈 3. Créer la stratégie de croisement de moyennes mobiles
# ----------------------------
class SMACrossStrategy(bt.Strategy):
    params = (
        ('sma_short', 20),  # période pour la moyenne mobile courte
        ('sma_long', 50),   # période pour la moyenne mobile longue
    )

    def __init__(self):
        # Calcul des deux moyennes mobiles
        self.sma_short = bt.ind.SMA(period=self.params.sma_short)
        self.sma_long = bt.ind.SMA(period=self.params.sma_long)

        # Détecter les croisements entre les deux SMA (cross up ou down)
        self.crossover = bt.ind.CrossOver(self.sma_short, self.sma_long)

    def next(self):
        # Cette méthode s’exécute à chaque nouvelle bougie (jour)
        if not self.position:
            # Si on n’a pas de position (ni achat, ni vente)
            if self.crossover > 0:
                self.buy()  # 🟢 Croisement haussier → on achète
        elif self.crossover < 0:
            self.sell()  # 🔴 Croisement baissier → on revend tout



# ----------------------------
# 4. Initialiser Backtrader et ajouter notre stratégie
# ----------------------------
cerebro = bt.Cerebro()  # Le moteur de backtest de Backtrader
cerebro.addstrategy(SMACrossStrategy)  # Ajout de notre stratégie dans le moteur

# Création du "data feed" à partir du DataFrame qu'on a préparé
data_feed = PandasData(dataname=data)

# Ajout des données au moteur de simulation
cerebro.adddata(data_feed)

# Définir le capital initial
cerebro.broker.set_cash(10000.0)

# Affichage du capital avant simulation
print(f"💰 Capital initial : {cerebro.broker.getvalue():.2f} $")

In [ ]:
# ----------------------------
# 5. Exécuter la simulation
# ----------------------------
cerebro.run()

# Affichage du capital après simulation
print(f" Capital final : {cerebro.broker.getvalue():.2f} $")


In [ ]:

# ----------------------------
# 6. Afficher le graphique des transactions
# ----------------------------
# Le graphique montrera :
# - le cours
# - les deux SMA
# - les trades (achats/ventes)
# Affichage propre
import matplotlib
# Activation du backend graphique pour VS Code

figs = cerebro.plot(iplot=True, style='candlestick', volume=True)
figs

In [ ]:
# Affichage du portefeuille final
portefeuille_final = cerebro.broker.getvalue()
print(f"💰 Portefeuille final : ${portefeuille_final:.2f}")

## Mesurer la performance de la stratégie

Après avoir simulé une stratégie de trading (comme le croisement de moyennes mobiles), il est essentiel d'évaluer sa performance. Voici trois indicateurs fondamentaux à analyser :

### 1. Capital final

Il s'agit de la valeur du portefeuille à la fin du backtest, en tenant compte :
- du cash restant,
- et de la valeur des positions en cours.

Cela permet de savoir si la stratégie a généré un gain ou une perte.

### 2. Rendement total (%)

C’est la variation du capital exprimée en pourcentage par rapport au capital initial.

Formule :

```
Rendement (%) = [(Capital final - Capital initial) / Capital initial] × 100
```

Un rendement positif indique une stratégie gagnante. Un rendement négatif signifie une perte nette.

### 3. Drawdown maximal

Le drawdown maximal mesure la plus forte baisse du portefeuille depuis un sommet. C’est un indicateur de risque.

Formule :

```
Drawdown = (Sommet du capital - creux suivant) / Sommet du capital
```

Une stratégie peut être rentable, mais risquée si le drawdown est élevé.

### Autres indicateurs possibles

- Taux de réussite des trades
- Nombre de trades gagnants/perdants
- Ratio gain/perte
- Durée moyenne des positions
- Sharpe Ratio (rendement ajusté au risque)

### Conclusion

Une bonne stratégie ne doit pas seulement être rentable, mais aussi stable et maîtrisée en termes de risque.


In [ ]:

# Récupérer le capital initial et final
capital_initial = 10000  # Ou la valeur que tu as utilisée dans set_cash
capital_final = cerebro.broker.getvalue()

# Calcul du rendement
rendement = (capital_final - capital_initial) / capital_initial * 100

print(f"Capital initial : {capital_initial:.2f} $")
print(f"Capital final   : {capital_final:.2f} $")
print(f"Rendement total : {rendement:.2f} %")


In [ ]:
# Option 1 : récupérer l'évolution du portefeuille à chaque step (nécessite une petite modification dans la stratégie)

# Adapter les données pour Backtrader
class PandasData(bt.feeds.PandasData):
    params = (
        ('datetime', 'Date'),
        ('open', 'open'),
        ('high', 'high'),
        ('low', 'low'),
        ('close', 'close'),
        ('volume', 'volume'),
        ('openinterest', -1),
    )

# Stratégie SMA croisée enrichie
class SMACross(bt.Strategy):
    def __init__(self):
        self.sma1 = bt.ind.SMA(period=20)
        self.sma2 = bt.ind.SMA(period=50)
        self.cross = bt.ind.CrossOver(self.sma1, self.sma2)
        self.portfolio_values = []
        self.trades = []

    def next(self):
        self.portfolio_values.append(self.broker.getvalue())

        if not self.position:
            if self.cross > 0:
                self.buy_price = self.data.close[0]
                self.buy()
        elif self.cross < 0:
            if self.position:
                sell_price = self.data.close[0]
                gain = sell_price - self.buy_price
                self.trades.append(gain)
                self.sell()

    def stop(self):
        # Capital
        capital_initial = cerebro.startingcash
        capital_final = self.broker.getvalue()
        rendement = (capital_final - capital_initial) / capital_initial * 100

        # Drawdown max
        peak = self.portfolio_values[0]
        max_dd = 0
        for val in self.portfolio_values:
            if val > peak:
                peak = val
            dd = (peak - val) / peak
            if dd > max_dd:
                max_dd = dd

        # Statistiques sur les trades
        total_trades = len(self.trades)
        winning_trades = len([t for t in self.trades if t > 0])
        losing_trades = total_trades - winning_trades
        win_rate = (winning_trades / total_trades) * 100 if total_trades > 0 else 0
        avg_gain = sum(self.trades) / total_trades if total_trades > 0 else 0
        gains = [t for t in self.trades if t > 0]
        losses = [t for t in self.trades if t < 0]
        avg_win = sum(gains) / len(gains) if gains else 0
        avg_loss = abs(sum(losses) / len(losses)) if losses else 0
        gain_loss_ratio = avg_win / avg_loss if avg_loss > 0 else 0

        # Affichage des résultats
        print("----- Résultats de la stratégie -----")
        print(f"Capital initial     : {capital_initial:.2f} $")
        print(f"Capital final       : {capital_final:.2f} $")
        print(f"Rendement total     : {rendement:.2f} %")
        print(f"Drawdown maximal    : {max_dd * 100:.2f} %")
        print(f"Nombre de trades    : {total_trades}")
        print(f"Trades gagnants     : {winning_trades}")
        print(f"Trades perdants     : {losing_trades}")
        print(f"Taux de réussite    : {win_rate:.2f} %")
        print(f"Gain moyen / trade  : {avg_gain:.2f} $")
        print(f"Ratio gain/perte    : {gain_loss_ratio:.2f}")

# Backtest
cerebro = bt.Cerebro()
cerebro.addstrategy(SMACross)
cerebro.adddata(PandasData(dataname=data))
cerebro.broker.setcash(10000.0)
cerebro.startingcash = 10000.0

cerebro.run()


## Défi Python : Implémentez une stratégie SMA(10) / SMA(30)

Créez une stratégie de backtesting basée sur un **croisement de moyennes mobiles simples** :

- Utilisez une **SMA courte de 10 jours**
- Utilisez une **SMA longue de 30 jours**
- Achetez lorsque la **SMA(10) croise au-dessus** de la SMA(30)
- Vendez lorsque la **SMA(10) croise en dessous** de la SMA(30)


In [ ]:

# Adapter les données pour Backtrader
class PandasData(bt.feeds.PandasData):
    params = (
        ('datetime', 'Date'),
        ('open', 'open'),
        ('high', 'high'),
        ('low', 'low'),
        ('close', 'close'),
        ('volume', 'volume'),
        ('openinterest', -1),
    )

# Stratégie SMA(10) / SMA(30)
class SMACross(bt.Strategy):
    def __init__(self):
        self.sma1 = bt.ind.SMA(period=10)  # SMA courte
        self.sma2 = bt.ind.SMA(period=30)  # SMA longue
        self.cross = bt.ind.CrossOver(self.sma1, self.sma2)
        self.portfolio_values = []
        self.trades = []

    def next(self):
        self.portfolio_values.append(self.broker.getvalue())

        if not self.position:
            if self.cross > 0:
                self.buy_price = self.data.close[0]
                self.buy()
        elif self.cross < 0:
            if self.position:
                sell_price = self.data.close[0]
                gain = sell_price - self.buy_price
                self.trades.append(gain)
                self.sell()

    def stop(self):
        capital_initial = cerebro.startingcash
        capital_final = self.broker.getvalue()
        rendement = (capital_final - capital_initial) / capital_initial * 100

        peak = self.portfolio_values[0]
        max_dd = 0
        for val in self.portfolio_values:
            if val > peak:
                peak = val
            dd = (peak - val) / peak
            if dd > max_dd:
                max_dd = dd

        total_trades = len(self.trades)
        winning_trades = len([t for t in self.trades if t > 0])
        losing_trades = total_trades - winning_trades
        win_rate = (winning_trades / total_trades) * 100 if total_trades > 0 else 0
        avg_gain = sum(self.trades) / total_trades if total_trades > 0 else 0
        gains = [t for t in self.trades if t > 0]
        losses = [t for t in self.trades if t < 0]
        avg_win = sum(gains) / len(gains) if gains else 0
        avg_loss = abs(sum(losses) / len(losses)) if losses else 0
        gain_loss_ratio = avg_win / avg_loss if avg_loss > 0 else 0

        print("----- Résultats de la stratégie SMA(10)/SMA(30) -----")
        print(f"Capital initial     : {capital_initial:.2f} $")
        print(f"Capital final       : {capital_final:.2f} $")
        print(f"Rendement total     : {rendement:.2f} %")
        print(f"Drawdown maximal    : {max_dd * 100:.2f} %")
        print(f"Nombre de trades    : {total_trades}")
        print(f"Trades gagnants     : {winning_trades}")
        print(f"Trades perdants     : {losing_trades}")
        print(f"Taux de réussite    : {win_rate:.2f} %")
        print(f"Gain moyen / trade  : {avg_gain:.2f} $")
        print(f"Ratio gain/perte    : {gain_loss_ratio:.2f}")

# Lancer le backtest
cerebro = bt.Cerebro()
cerebro.addstrategy(SMACross)
cerebro.adddata(PandasData(dataname=data))
cerebro.broker.setcash(10000.0)
cerebro.startingcash = 10000.0

cerebro.run()
cerebro.plot(iplot=False, style='candlestick', volume=True)